In [1]:
# Basic
from collections import Counter
import math
import numpy as np
import os
import pprint

# yaml specific
import yaml

# Data handling
import pandas as pd
from tqdm import tqdm

3

In [2]:
# Config variables
raw_data_path = "raw_data"
clean_data_path = "clean_data"

### Utility functions. 

In [3]:
def get_clean_teams(major_teams):
    # remove bad chars from string
    bad_chars = [';', ':', '!', "*", "'", '"', '[', ']', ',']
    teams = []
    for team in major_teams.split(", "):
        for char in bad_chars:
            team = team.replace(char, "")
        teams.append(team.strip().rstrip())
    return teams

# This notebook will be used to curate 4 tables. These tables are independent of matches
## 1) Country 2) Tournament 3) Team 4) Player
### Initially handling everything through CSVs. Later, we will see if using a DB is required

In [4]:
players = pd.read_csv("raw_data/players.csv")
players = players.loc[:, ~players.columns.str.contains('^Unnamed')]

C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (101,107,108,140,141,145,146,147,153,154,158,159,160) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Country table

### Focusing only on the top 20 countries with professional cricket players

In [5]:
countries = []
for key, value in players["COUNTRY"].value_counts().nlargest(20).iteritems():
    countries.append(key)

In [6]:
country_columns = ["country_id", "country_name"]
country_csv = pd.DataFrame(columns = country_columns)
for i in range(len(countries)):
    country_id = i+1
    country_name = countries[i]
    country_csv = country_csv.append({"country_id" : country_id, 
                                      "country_name" : country_name}, ignore_index=True)
country_csv.to_csv(os.path.join(clean_data_path, "country.csv"), index=False)

## Tournament table

### Each zip file in https://cricsheet.org/ is going to be considered as a separate tournament. This list is going to be manually curated due to its short size

In [7]:
tournaments = [
    {
        "tournament_name" : "TEST",
        "tournament_format" : "TEST",
        "tournament_full_name" : "Test International Match"
    },
    {
        "tournament_name" : "ODI",
        "tournament_format" : "ODI",
        "tournament_full_name" : "ODI Match"
    },
    {
        "tournament_name" : "T20I",
        "tournament_format" : "T20",
        "tournament_full_name" : "T20 International Match"
    },
    {
        "tournament_name" : "BBL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Big Bash League"
    },
    {
        "tournament_name" : "IPL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Indian Premier Leargue"
    },
    {
        "tournament_name" : "CPL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Caribbean Premier Leargue"
    },
    {
        "tournament_name" : "T20B",
        "tournament_format" : "T20",
        "tournament_full_name" : "T20 Blast"
    },
    {
        "tournament_name" : "PSL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Pakistan Super League"
    },
    {
        "tournament_name" : "SSM",
        "tournament_format" : "T20",
        "tournament_full_name" : "Super Smash"
    },
    {
        "tournament_name" : "WBB",
        "tournament_format" : "T20",
        "tournament_full_name" : "Womens Big Bash League"
    }
]

In [8]:
tournament_columns = ["tournament_id", "tournament_name", "tournament_format", "tournament_full_name"]
tournament_csv = pd.DataFrame(columns = tournament_columns)
for i in range(len(tournaments)):
    tournament_id = i+1
    tournament_name = tournaments[i]["tournament_name"]
    tournament_format = tournaments[i]["tournament_format"]
    tournament_full_name = tournaments[i]["tournament_full_name"]
    tournament_csv = tournament_csv.append({"tournament_id" : tournament_id, 
                                      "tournament_name" : tournament_name,
                                      "tournament_format" : tournament_format, 
                                      "tournament_full_name" : tournament_full_name}, ignore_index=True)
tournament_csv.to_csv(os.path.join(clean_data_path, "tournament.csv"), index=False)

## Team table


### Considering only those teams where the players belong to the above 20 countries and aged < 50 (as of 2019)

In [9]:
# considering only the top 10 countries as the remaining countries barely have 10 important players that can be seperately ingested. 
required_countries = countries[:-10]
required_columns = ['NAME', 'COUNTRY', 'Full name', 'Birthdate', 'Major teams', 'Batting style', 'Bowling style', 'Other']
required_players = players[(players["COUNTRY"].isin(countries)) & (players["Age"] <= 55)][required_columns]

In [10]:
teams = []
for index, row in tqdm(required_players.iterrows()):
    # skip columns that are not string
    if type(row["Major teams"]) is not str:
        continue
    for team in get_clean_teams(row["Major teams"]):
        teams.append(team)

38017it [00:02, 14630.17it/s]


### If a team has less than 'threshold' registered professional players, ignore that team

In [11]:
threshold = 25
threshold_count = 0
teams_set = set()
for team, count in Counter(teams).most_common():
    if count > threshold:
        threshold_count += 1
        teams_set.add(team)
teams_list = list(teams_set)

In [12]:
len(teams_list)

599

In [13]:
team_columns = ["team_id", "team_name", "team_label"]
team_csv = pd.DataFrame(columns = team_columns)
for i in tqdm(range(len(teams_list))):
    team_id = i+1
    team_name = teams_list[i]
    team_label = "".join(word[0] for word in team_name.split(" "))
    team_csv = team_csv.append({"team_id" : team_id, 
                                "team_name" : team_name, 
                                "team_label" : team_label}, ignore_index=True)
team_csv.to_csv(os.path.join(clean_data_path, "team.csv"), index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 599/599 [00:01<00:00, 469.66it/s]


## Player table

### Considering only those players who belong to the 20 countries and aged < 50 (as of 2019)

### Utility maps to ingest players

In [14]:
df_country = pd.read_csv(os.path.join(clean_data_path, "country.csv"))
df_country = df_country.loc[:, ~df_country.columns.str.contains('^Unnamed')]
country_id_map = dict(zip(df_country.country_name, df_country.country_id))

In [15]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

In [16]:
player_columns = ["player_id", "player_display_name", "player_name", "player_full_name", "batting_style", "bowling_style", "birthdate", "country_id"]
player_csv = {}
i = 0 # player tracker
for index, row in tqdm(required_players.iterrows()):
    
    # If player doesnt have a proper team skip
    if type(row["Major teams"]) is not str:
        continue
    
    # Getting the list of team ids this player has played in
    team_list_ids = ""
    for team in get_clean_teams(row["Major teams"]):
        if team in team_id_map:
            team_list_ids += str(team_id_map[team]) + ","
            
    # If the player doesnt play in atleast 2 teams skip them
    if len(team_list_ids.split(",")) < 3 :
        continue
        
    # removing last comma
    team_list_ids = team_list_ids[:-1]
    
    player_id = i+1
    player_display_name = "" # This needs to be updated later with fuzzy matching
    player_name = row["NAME"]
    player_full_name = row["Full name"]
    batting_style = row["Batting style"]
    bowling_style = row["Bowling style"]
    birthdate = row["Birthdate"]
    country_id = country_id_map[row["COUNTRY"]]
    list_team_ids = team_list_ids
    
    player_csv[i] = {
                        "player_id" : player_id, 
                        "player_display_name" : player_display_name,
                        "player_name" : player_name,
                        "player_full_name" : player_full_name,
                        "batting_style" : batting_style,
                        "bowling_style" : bowling_style,
                        "birthdate" : birthdate,
                        "country_id" : country_id,
                        "list_team_ids" : list_team_ids
    }
    
    i += 1
    
df_player_csv = pd.DataFrame.from_dict(player_csv, "index")
df_player_csv.to_csv(os.path.join(clean_data_path, "player.csv"), index=False)

38017it [00:03, 12435.55it/s]
